In [1]:
import numpy as np
from datetime import datetime, date, timedelta
import pandas as pd

In [2]:
class bonds:
        
    def __init__(self,
                 N : np.float,
                 tf : np.float,
                 tr : np.float,
                 d_emis : np.str,
                 d_jouis : np.str,
                 d_ech : np.str,
                 d_val : np.str):  
    
        self.N = N
        self.tf = tf
        self.tr = tr
        self.d_emis = datetime.strptime(d_emis, '%d/%m/%Y').date()
        self.d_jouis = datetime.strptime(d_jouis, '%d/%m/%Y').date()
        self.d_ech = datetime.strptime(d_ech, '%d/%m/%Y').date()
        self.d_val = datetime.strptime(d_val, '%d/%m/%Y').date()
           
        "sortie"
        
        self.dirty_price = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[0]
        self.coupon_couru = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[1]
        self.clean_price = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[2]
        self.coupon_recu = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[3]
        self.duration = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[4]
        self.sensibilite = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[5]
        self.convexite = self.calculator(N,tf,tr,d_emis,d_jouis,d_ech,d_val)[6]
    
    def calculator(self,N,tf,tr,d_emis,d_jouis,d_ech,d_val): #7 parametres dont 4 dates
        
        "calcul intermediaire"
        
        # maturite residuelle et initiale
        self.mr = (self.d_ech - self.d_val).days
        self.mi = (self.d_ech - self.d_emis).days
        
        # date du dernier coupon (ddc) et date du prochain coupon (dpc)
        ddc = self.d_jouis
        dpc = ddc.replace(year = ddc.year+1)
        while dpc < self.d_val:
            dpc = dpc.replace(year= dpc.year +1)
        self.ddc = dpc.replace(year = dpc.year-1)
        self.dpc = dpc
        
        # Nombre de jours exact de l’année écoulée ???????
        self.A = 365
        
        # nombre de jours entre date de valorisation et date du prochain coupon 
        self.nj = (self.dpc - self.d_val).days 
        
        # nombre de coupon restant avant la maturité
        self.n = (self.d_ech.year - self.dpc.year) + 1
        
        # nombre de coupon deja recu avant la date de valorisation
        self.nc = (self.ddc.year - self.d_jouis.year) + 1
        
        # Date du premier coupon ??
        # d_pc = "date_jouissnce+ frequence"???

        if self.mi <= 365 : #maturite initiale inferieure a 1an

            C = N * tf * self.mi / 360
            Cr = C*self.nc
            P = (N + C) / (1+ tr*self.mr /360)
            Cc = N*tf* (self.mi-self.mr) / 360
            
            
            # Duration et sensiblite
            num = 0
            D = 0
            for i in range(1,self.n+1):
                num = num + (self.nj/self.A + i-1)*C
            num = num + (self.nj/self.A + self.n-1) * N
            D = num/ P
            S = D/(1+self.tr)
                    
            # Convexite
            Co = 0
            for i in range(1,self.n+1):
                num = num + C*(self.nj/self.A + i-1)* (self.nj/self.A + i) / (1+self.tr)**(self.nj/self.A + i-1)
            num = num + N * (self.nj/self.A + self.n-1)* (self.nj/self.A + self.n) / (1+self.tr)**(self.nj/self.A + self.n-1)
            Co = num/ (P*(1+self.tr)**2)              

        else:  #maturite initiale superieure a 1an

            if self.mr <= 365: #maturite residuelle inferieure a 1an

                    C = N*tf 
                    Cr = C*self.nc
                    P = (N + C) / (1+ tr*self.mr/360)
                    Cc = N*tf* (self.d_val-self.ddc).days / self.A
                    
                    # Duration et sensiblite
                    num = 0
                    D = 0
                    for i in range(1,self.n+1):
                        num = num + (self.nj/self.A + i-1)*C
                    num = num + (self.nj/self.A + self.n-1) * N
                    D = num/ P
                    S = D/(1+self.tr)
                    
                    # Convexite
                    Co = 0
                    for i in range(1,self.n+1):
                        num = num + C*(self.nj/self.A + i-1)* (self.nj/self.A + i) / (1+self.tr)**(self.nj/self.A + i-1)
                    num = num + N * (self.nj/self.A + self.n-1)* (self.nj/self.A + self.n) / (1+self.tr)**(self.nj/self.A + self.n-1)
                    Co = num/ (P*(1+self.tr)**2)

            else: # maturite residuelle superieure a 1an

                    C = N*tf
                    Cr = C*self.nc
                    Cc = N*tf* (self.d_val-self.ddc).days / self.A
                    s = 0
                    for i in range(1,self.n+1): # n compris
                        s = s + C/ (1+tr)**(i-1)
                    P = ( s + N/ (1+tr)**(self.n-1))  / (1+tr)**(self.nj / self.A)
                    
                    # Duration et sensiblite
                    num = 0
                    D = 0
                    for i in range(1,self.n+1):
                        num = num + C *( self.nj/self.A + i-1)
                    num = num + N*( self.nj/self.A + self.n-1)
                    D = num/ P
                    S = D/(1+self.tr)
                    
                    # Convexite
                    Co = 0
                    for i in range(1,self.n+1):
                        num = num + C*(self.nj/self.A + i-1)* (self.nj/self.A + i) / (1+self.tr)**(self.nj/self.A + i-1)
                    num = num + N * (self.nj/self.A + self.n-1)* (self.nj/self.A + self.n) / (1+self.tr)**(self.nj/self.A + self.n-1)
                    Co = num/ (P*(1+self.tr)**2)

                        
        return  (round(P),round(Cc),round(P-Cc),round(Cr), round(D,2), round(S,2), round(Co,2))
    

## Example

In [3]:
N = 100000
tf = 0.061
tr = 0.01766
d_emis = '05/04/2004'
d_jouis = '05/04/2004'
d_ech = '05/04/2024'
d_val = "08/06/2021"
bond1 = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val)
print('dirty_price = ', bond1.dirty_price)
print('clean_price = ', bond1.clean_price)
print('duration = ', bond1.duration)
print('Sensibiliite = ', bond1.sensibilite)
print('Convexite = ', bond1.convexite)

dirty_price =  112902
clean_price =  111832
duration =  2.8
Sensibiliite =  2.75
Convexite =  12.37


In [4]:
(bond1.nj/365)*6100

5030.41095890411

In [5]:
(bond1.nj/365+1)*6100

11130.410958904109

In [6]:
(bond1.nj/365+2)*106100

299696.16438356164

In [7]:
(5030.41095890411+ 11130.410958904109+ 299696.16438356164)/112902

2.7976208242667964

In [18]:
N = 100000
tf = bond['Cpn Rate'][10]/100
tr = bond['Yield Mat'][10]/100
d_emis = bond['Date emission'][10].strftime("%d/%m/%Y")
d_jouis = bond['Jouissance'][10].strftime("%d/%m/%Y")
d_ech = bond['Mat. Date'][10].strftime("%d/%m/%Y")
d_val = "08/06/2021"
bond2 = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val)
print('dirty_price = ', bond2.dirty_price)
print('clean_price = ', bond2.clean_price)

dirty_price =  102569
clean_price =  101371


## Real Data

In [11]:
import os
os.getcwd()
os.chdir("C:\\Users\\Halem\\Downloads\\final")

In [12]:
pd.set_option("display.max_row",100)
pd.set_option("display.max_column",100)
bond = pd.read_excel("ptfbonds.xlsx") 

In [13]:
#bond.head()

In [14]:
data = pd.DataFrame( {'Short Name' : bond['Short Name'],'Clean_price' : 1000*bond['Clean Price'], 
                      'Price' : None, 'Ecart' : None , 'Duration' : bond['Dur.'], 'Dur_th' : None, 'Ecart_dur' : None ,
                     'Sensibilite' : bond['Mod Dur'], 'Sen_th' : None, 'Ecart_sens' : None ,
                     'Convexite' : bond['Cnvx.'], 'Con_th' : None, 'Ecart_con' : None } )

In [15]:
N = 100000
for i in range (44): 
    tf = bond['Cpn Rate'][i]/100
    tr = bond['Yield Mat'][i]/100
    d_emis = bond['Date emission'][i].strftime("%d/%m/%Y")
    d_jouis = bond['Jouissance'][i].strftime("%d/%m/%Y")
    d_ech = bond['Mat. Date'][i].strftime("%d/%m/%Y")
    d_val = "08/06/2021"
    
    data['Price'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).clean_price
    data['Ecart'][i] = data['Price'][i]-data['Clean_price'][i]
    
    data['Dur_th'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).duration
    data['Ecart_dur'][i] = data['Dur_th'][i]-data['Duration'][i]
    
    data['Sen_th'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).sensibilite
    data['Ecart_sens'][i] = data['Sen_th'][i]-data['Sensibilite'][i]
    
    data['Con_th'][i] = bonds(N,tf,tr,d_emis,d_jouis,d_ech,d_val).convexite
    data['Ecart_con'][i] = data['Con_th'][i]-data['Convexite'][i]


In [16]:
data.iloc[0:44,:]

,Short Name,Clean_price,Price,Ecart,Duration,Dur_th,Ecart_dur,Sensibilite,Sen_th,Ecart_sens,Convexite,Con_th,Ecart_con
0,MA0002007096,111832,111832,0,2.67,2.8,0.13,2.62,2.75,0.13,9.668,12.37,2.702
1,MA0002007641,116610,116608,-2,3.84,4.15,0.31,3.77,4.08,0.31,19.068,23.07,4.002
2,MA0002008128,101404,101404,0,0.67,0.67,0,0.67,0.66,-0.01,0.886,1.72,0.834
3,MA0002009936,108384,108385,1,3.5,3.75,0.25,3.44,3.68,0.24,15.676,19.29,3.614
4,MA0002010934,112970,112971,1,5.31,5.94,0.63,5.2,5.83,0.63,33.829,39.54,5.711
5,MA0002011114,102255,102255,0,0.87,0.87,0,0.86,0.86,0,1.482,2.4,0.918
6,MA0002012369,120616,120616,0,6.06,6.99,0.93,5.93,6.84,0.91,45.253,51.96,6.707
7,MA0002012922,107449,107448,-1,2.02,2.09,0.07,1.99,2.06,0.07,6.101,8.12,2.019
8,MA0002013441,135677,135679,2,10.36,14.12,3.76,10.11,13.77,3.66,130.771,144.2,13.429
9,MA0002013797,114208,114209,1,7.72,9.37,1.65,7.55,9.17,1.62,71.254,80.22,8.966


In [ ]:
"""
df1 = data['Ecart']
df1.to_csv("C:\\Users\\Halem\\Downloads\\final\\remi1.csv", header=True, sep='\n')

df2 = data['Ecart_dur']
df2.to_csv("C:\\Users\\Halem\\Downloads\\final\\remi2.csv", header=True, sep='\n')

df3 = data['Ecart_con']
df3.to_csv("C:\\Users\\Halem\\Downloads\\final\\remi3.csv", header=True, sep='\n')
"""